# HOW TO PREDICT AN EMPLOYEE'S RESIGNATION ?

Nowadays, many companies have to deal with attrition. It is possible to find on Internet the following definition for this concept :

"The unpredictable and uncontrollable, but normal, reduction of work force due to resignations, retirement, sickness, or death." (http://www.businessdictionary.com/definition/attrition.html)

But is it really unpredictable ? The famous data-science oriented website Kaggle provides us an useful dataset (unfortunately not available anymore) for HR Analytics. This dataset contains information (such as evaluation, wage level or department) about 15 000 employees. My goal is to anticipate the resignation of the best of them.

## Data preprocessing

In [1]:
import pandas
import pandas
import matplotlib.pyplot as plt
import numpy as np
import math
#import sklearn.cross_validation
#import sklearn.grid_search
import random
from sklearn.linear_model import LogisticRegression
import sklearn.feature_selection
import sklearn.ensemble
hr = pandas.read_csv("C:/Users/tbeucher/Documents/Projet Python/HR_comma_sep.csv")
hr.info()
hr.head(n = 10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
satisfaction_level       14999 non-null float64
last_evaluation          14999 non-null float64
number_project           14999 non-null int64
average_montly_hours     14999 non-null int64
time_spend_company       14999 non-null int64
Work_accident            14999 non-null int64
left                     14999 non-null int64
promotion_last_5years    14999 non-null int64
sales                    14999 non-null object
salary                   14999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low
5,0.41,0.50,2,153,3,0,1,0,sales,low
6,0.10,0.77,6,247,4,0,1,0,sales,low
7,0.92,0.85,5,259,5,0,1,0,sales,low
8,0.89,1.00,5,224,5,0,1,0,sales,low
9,0.42,0.53,2,142,3,0,1,0,sales,low


#### Selection of the best employees

I want to keep only the best employees because an company does not need to retain the bad ones. Arbitrarily, a good employee is someone who satisfies ONE of the following conditions :
- His/her last evaluation is above average (70% is the mean for this variable in the whole dataset)
- He/she has been given a promotion recently (last 5 years)
- He/she is an hard-worker (works more than 200h/a month)

In [3]:
#The dataset with all "good" employees will be called hr_good

good = hr.apply ( lambda row : row["last_evaluation"] > 0.70 or row["average_montly_hours"] > 200 
                 or row["promotion_last_5years"] > 0, axis = 1 )
hr_good = hr[good]

print("We have", len(hr_good), "observations in the new dataset")
print(int(np.mean(hr_good.left)*100), "% of these employees left the company")


We have 10480 observations in the new dataset
19 % of these employees left the company


#### Train/test split

In [4]:
split = hr_good.apply ( lambda row : random.randint(0,10000), axis = 1 ) 
hr_good ["split"] = split

cond_test = hr_good.apply ( lambda row : row["split"] > 9850, axis = 1 ) #around 150 employees in the test dataset
cond_train = hr_good.apply( lambda row : row["split"] < 9851, axis = 1 )

test = hr_good [cond_test]
hr_good = hr_good[cond_train]

test = test.drop("split", axis = 1)
hr_good = hr_good.drop("split", axis = 1)
copie_hr = hr #just for safety
print("There are", len(test), "good employees in the test dataset.")

C:\Users\tbeucher\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


There are  153 good employees in the test dataset.


Adding set.seed, train/test split sklearn

## Data Analysis

To be improved

In [9]:
left = hr_good [ hr_good.left > 0 ]
stay = hr_good [ hr_good.left < 1 ]
print("Summary of quantitative variables (only on employees who left the company)")
print("Theses employees are", len(left), ". That represents approximately", int(len(left)/len(hr_good)*100), "% of the train dataset")
pandas.DataFrame.describe(left)

Summary of quantitative variables (only on employees who left the company)
Theses employees are 1985 . That represents approximately 19 % of the train dataset


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,1985.000000,1985.000000,1985.000000,1985.000000,1985.000000,1985.000000,1985.0,1985.000000
mean,0.463592,0.875914,5.218640,256.318892,4.521914,0.047355,1.0,0.009572
std,0.345975,0.103355,1.168537,32.735292,0.821000,0.212451,0.0,0.097391
min,0.090000,0.450000,2.000000,128.000000,2.000000,0.000000,1.0,0.000000
25%,0.100000,0.830000,4.000000,242.000000,4.000000,0.000000,1.0,0.000000
50%,0.480000,0.890000,5.000000,258.000000,5.000000,0.000000,1.0,0.000000
75%,0.810000,0.950000,6.000000,277.000000,5.000000,0.000000,1.0,0.000000
max,0.920000,1.000000,7.000000,310.000000,6.000000,1.000000,1.0,1.000000


In [10]:
print("Summary of quantitative variables (only on employees who are still working for the company)")
print("Theses employees are", len(stay), ". That represents approximately", int(len(stay)/len(hr_good)*100),"% de la base de données")
pandas.DataFrame.describe(stay)

Summary of quantitative variables (only on employees who are still working for the company)
Theses employees are 8342 . That represents approximately 80 % de la base de données


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,8342.000000,8342.000000,8342.000000,8342.000000,8342.000000,8342.000000,8342.0,8342.000000
mean,0.677281,0.766845,3.824982,212.943299,3.391153,0.172980,0.0,0.035723
std,0.215630,0.150735,0.962956,42.979044,1.571944,0.378253,0.0,0.185609
min,0.120000,0.360000,2.000000,96.000000,2.000000,0.000000,0.0,0.000000
25%,0.550000,0.660000,3.000000,181.000000,2.000000,0.000000,0.0,0.000000
50%,0.700000,0.780000,4.000000,219.000000,3.000000,0.000000,0.0,0.000000
75%,0.840000,0.890000,4.000000,248.000000,4.000000,0.000000,0.0,0.000000
max,1.000000,1.000000,6.000000,287.000000,10.000000,1.000000,0.0,1.000000
